# Predict Bankruptcy

### 1) Load required modules

In [1]:
import pandas                   as     pd
import numpy                    as     np
import seaborn                  as     sns
import matplotlib.pyplot        as     plt
import statsmodels.api          as     sm
from   sklearn.preprocessing    import OrdinalEncoder
from   sklearn                  import metrics
from   sklearn                  import tree
from   sklearn.model_selection  import train_test_split
import sys, os
import os


In [2]:
# Parent Directory path
parent_dir = r'D:\DrPKV\20220901'
os.chdir(parent_dir)

In [3]:
import time
from    datetime   import datetime
from    datetime   import timedelta

In [4]:
### Modules for imputing
from sklearn.experimental  import   enable_iterative_imputer
from sklearn.impute        import   IterativeImputer
from sklearn.linear_model   import   LinearRegression


In [5]:
### Explore the algorithm wrapped by RFE
###
from numpy                         import mean
from numpy                         import std
from sklearn.datasets              import make_classification
from sklearn.model_selection       import cross_val_score
from sklearn.model_selection       import RepeatedStratifiedKFold
from sklearn.feature_selection     import RFE
from sklearn.feature_selection     import RFECV
###
from sklearn.linear_model          import LogisticRegression
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.svm                   import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.pipeline              import Pipeline
from matplotlib                    import pyplot as plt
###

In [6]:
pd.set_option('display.float_format', lambda x: '%.5f' % x) # To get rid of E notation

### Define required functions

####  Function to detect zero & null values and report column-wise count & percentage of zero & missing values

In [7]:
"""
Function name : missing_zero_values_table
Arguments:
Input: Dataset name
Output : Report on column-wise count & percentage of zero & missing values

"""
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero & Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero & Missing Values'] = 100 * mz_table['Total Zero & Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

In [8]:
### ------- -------------- -------------- -----------
### get a list of models to evaluate
### ------- -------------- -------------- -----------
def get_models():
    models = dict()
    # 1) Logistic Regression
    rfe            = RFE(estimator=LogisticRegression(), n_features_to_select = 10)
    model          = DecisionTreeClassifier()
    models['lr']   = Pipeline(steps=[('s',rfe),('m',model)])
    # 2) CART / Decision Tree
    rfe            = RFE(estimator=DecisionTreeClassifier(), n_features_to_select = 10)
    model          = DecisionTreeClassifier()
    models['cart'] = Pipeline(steps=[('s',rfe),('m',model)])
    # 3) Random Forest
    rfe           = RFE(estimator=RandomForestClassifier(), n_features_to_select = 10)
    model         = RandomForestClassifier()
    models['rf']  = Pipeline(steps=[('s',rfe),('m',model)])
    # 4) SVC
    rfe           = RFE(estimator=SVC(), n_features_to_select = 10)
    model         = SVC()
    models['SVM'] = Pipeline(steps=[('s',rfe),('m',model)])
    # 5) LDA
    rfe           = RFE(estimator=LinearDiscriminantAnalysis(), n_features_to_select = 10)
    model         = LinearDiscriminantAnalysis()
    models['LDA'] = Pipeline(steps=[('s',rfe),('m',model)])    
    # 6) KNN
    rfe           = RFE(estimator=KNeighborsClassifier(), n_features_to_select = 10)
    model         = KNeighborsClassifier()
    models['LDA'] = Pipeline(steps=[('s',rfe),('m',model)])        
    
    return models
 

In [9]:
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='recall', cv=cv, n_jobs=-1)
    return scores

### 2) Read dataset from an Excel file

In [10]:
file_name   = './Data/BankruptcyData.xlsx'

In [11]:
df_Orig     =   pd.read_excel(file_name, sheet_name = 'Full data')

In [12]:
df_Orig.info() # to know the structure of dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 34 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Row                                       9000 non-null   int64  
 1   Year                                      9000 non-null   int64  
 2   Company_name                              9000 non-null   object 
 3   Year Encoded                              9000 non-null   int64  
 4    Dummy Coded :Healthy=1; NPA= 0           9000 non-null   int64  
 5   Asset_turnover                            8299 non-null   float64
 6   Receivable_turnover(new)                  7390 non-null   float64
 7   Inventory_turnover                        5702 non-null   float64
 8   Cash_ratio                                8278 non-null   float64
 9   Quick_ratio                               8278 non-null   float64
 10  Current_ratio                       

### Observations

**We have 9000 observations and 34 columns in this dataset**

**We find from the data dictionary, the column names and description.**

|S.No | Column name | Description |
|--- | :------------------- |:-----------------------------------| 
| 1 | Row | Company code |
| 2 | Year	| year |
| 3 | Company_name | Company Name |
| 4 | Year Encoded	| year; 0 means latest year |
| 5 | Dummy Coded :Healthy=1; NPA= 0 | NPA = 0 and 1= Healthy, Our Target variable |
| 6 | Asset_turnover | Total income/ Total assets |
| 7 | Receivable_turnover(new) | Net sales/Total assets |
| 8 | Inventory_turnover | 	COGS/ Total inventories |
| 9 | Cash_ratio | Cash and cash balance/ Total Current liabilities | 
| 10 | Quick_ratio | Cash and Cash Equivalents + Receivables + Marketable securities/T Total Current liabilities | 
| 11 | Current_ratio | Current asset/current liabilities | 
| 12 | ROA(new) | Net income/Total assets | 
| 13 | ROE(new) | Net income/Shareholder's equity | 
| 14 | ROS(new) | Net income/Total sales | 
| 15 | ROI(new) | Net income/Total investment | 
| 16 | debt_asset | Total debt/Total asset | 
| 17 | debt_equity | Total debt/ Total equity | 
| 18 | debt_income | Total debt / EBIT | 
| 19 | Interest_coverage | EBITDA/Interest | 
| 20 | Asset_coverage | Total asset - (CA-CL) / (Total debt) | 
| 21 | EBIT_Sales | EBIT/Total sales | 
| 22 | Sales_CE | Sales/Total capital employed | 
| 23 | ROCE_CE | (EBIT/Sales) * (Sales/CE) | 
| 24 | Changeinsales_Industry | Sales (current year)- Sales (Previous year)/ Sales (current year) | 
| 25 | Grossvaluedadded | Grossvaluedadded/Total grossvaluedadded | 
| 26 | Ln_GVA | Ln (Gross value added) | 
| 27 | Operating Cash Flow/Total Sales | Operating cash flow/Total sales | 
| 28 | Operating Cash Flow/Total Debt | Operating cash flow/Total debt | 
| 29 | Operating Cash Flow/Shareholder's Equity | Operating cash flow /Total equity | 
| 30 | Fixed Asset Turnover Ratio | Total income/ Fixed asset | 
| 31 | YOY Sales Growth Rate | Y-O-Y Sales Growth rate  | 
| 32 | YOY EBIT Growth Rate | Y-O-Y EBIT Growth rate  | 
| 33 | Total shareholders' funds | Total shareholder's equity | 
| 34 | Shareholderquity_code | Dummy variable 	0 = healthy firm +ve equity,	1 = healthy firm -ve equity,2 = banktrupt firm +ve, equity,	3 = banktrupt firm -ve equity |

We observe that the following columns are not required
* 1) Row representing the company code
* 2) Company_name
- We are more interested in knowing the characteristics of the company going bankrupt rather than the company name
* 3) Year
* 4) Shareholderquity_code
- This is very similar to the target variable revealing whether the company is healthy or NPA
We already have the column Year Encoded representing the year. So we remove one of them and hence Year is removed

In [13]:
df              =   df_Orig.copy()

In [14]:
unwanted_cols   =   ['Row', 'Year', 'Company_name']

# We are including the variable,'Shareholderquity_code' 

### We want the variable, Shareholderquity_code only for comparing with the target variable and however, we are not gping to 
### use this in model building

df.drop(unwanted_cols, axis = 1, inplace = True)

### Rename the target variable, "  Dummy Coded :Healthy=1; NPA= 0" as "Target"

In [15]:
colnames =  df.columns
colnames

Index(['Year Encoded', ' Dummy Coded :Healthy=1; NPA= 0', 'Asset_turnover',
       'Receivable_turnover(new)', 'Inventory_turnover', 'Cash_ratio',
       'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)', 'ROS(new)',
       'ROI(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       'Operating Cash Flow/Shareholder's Equity',
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', 'Total shareholders' funds',
       'Shareholderquity_code'],
      dtype='object')

In [16]:
new_colnames  =  ['Year Encoded', 'Target', 'Asset_turnover',
       'Receivable_turnover(new)', 'Inventory_turnover', 'Cash_ratio',
       'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)', 'ROS(new)',
       'ROI(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       "Operating Cash Flow/Shareholder's Equity",
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', "Total shareholders' funds", 'Shareholderquity_code']

In [17]:
df.columns = new_colnames

### Interchange the class labels for the target variable to make 1 for NPA

In [18]:
df['Target'].replace(0,2, inplace =True)
df['Target'].replace(1,0, inplace =True)
df['Target'].replace(2,1, inplace =True)
df.tail().T

,8995,8996,8997,8998,8999
Year Encoded,-4.00000,-3.00000,-2.00000,-1.00000,0.00000
Target,1.00000,1.00000,1.00000,1.00000,1.00000
Asset_turnover,0.00887,0.18245,0.00003,0.00018,0.00000
Receivable_turnover(new),0.03602,0.74623,0.00012,0.00075,0.00000
Inventory_turnover,NaN,NaN,NaN,NaN,0.00000
Cash_ratio,0.00040,0.00025,0.00005,0.00012,-0.00006
Quick_ratio,0.00040,0.00025,0.00005,0.00012,-0.10557
Current_ratio,1.69937,1.71666,1.68236,2.00460,1.27291
ROA(new),-0.00793,-0.00885,-0.00989,-0.07698,-0.00760
ROE(new),0.13771,0.17965,0.24767,2.70893,-0.30925


#### 3.2)  To know about missing values - Column-wise  count & Percentage of missing values 

In [19]:
missing_zero_values_table(df)

Your selected dataframe has 31 columns and 9000 Rows.
There are 29 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero & Missing Values,% Total Zero & Missing Values,Data Type
ROI(new),25,4132,45.90000,4157,46.20000,float64
Shareholderquity_code,3305,3790,42.10000,7095,78.80000,float64
Inventory_turnover,4364,3298,36.60000,7662,85.10000,float64
Receivable_turnover(new),401,1610,17.90000,2011,22.30000,float64
Interest_coverage,218,1569,17.40000,1787,19.90000,float64
Operating Cash Flow/Total Debt,15,1306,14.50000,1321,14.70000,float64
Asset_coverage,0,1306,14.50000,1306,14.50000,float64
ROCE_CE,70,1258,14.00000,1328,14.80000,float64
Fixed Asset Turnover Ratio,476,1007,11.20000,1483,16.50000,float64
Cash_ratio,21,722,8.00000,743,8.30000,float64


### Observations

* We observe that there are 29 columns that have missing values. We do not want to incur data loss by removing rows with missing values. Let us impute them.

* Detecting and handling missing values in the correct way is important, as they can impact the results of the analysis. It cannot be imputed with general ways of using mean, mode, or median which ignores the inherent relationship among data and also it can pollute the data. 

* We observe that on a few occasions, data is missing in a dataset and is related to the other features and hence they can be predicted using other feature values.  Imputing by prediction of missing values is superior to other techniques since the inherent relationship among data is not ignored.

* We are imputing missing numerical values using the IterativeImputer class in sklearn. 


Ref: https://www.numpyninja.com/post/mice-and-knn-missing-value-imputations-through-python



In [20]:
df['Shareholderquity_code'].value_counts()

0.00000    3305
2.00000     865
1.00000     601
3.00000     439
Name: Shareholderquity_code, dtype: int64

### Observations

There are 3305 missing values in the column,Shareholderquity_code.

#### 3.3)  Imputation of missing values through Multiple Imputation by Chained Equation

In [21]:
lreg         =  LinearRegression()
imp          =  IterativeImputer(estimator= lreg, missing_values = np.nan, max_iter = 10, verbose = 2,\
                                 imputation_order= 'roman',random_state = 0)
X            =  imp.fit_transform(df)

[IterativeImputer] Completing matrix with shape (9000, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.20
[IterativeImputer] Change: 12802672566.19751, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.37
[IterativeImputer] Change: 2504009239.771635, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.55
[IterativeImputer] Change: 2375127160.0775986, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.74
[IterativeImputer] Change: 1492836889.0066626, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.91
[IterativeImputer] Change: 1545497817.705276, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 6/10, elapsed time 1.12
[IterativeImputer] Change: 1344871520.1793363, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 7/10, elapsed time 1.31
[Iterat

C:\Users\User\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [22]:
X_df    =    pd.DataFrame(X, columns = df.columns)
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year Encoded                              9000 non-null   float64
 1   Target                                    9000 non-null   float64
 2   Asset_turnover                            9000 non-null   float64
 3   Receivable_turnover(new)                  9000 non-null   float64
 4   Inventory_turnover                        9000 non-null   float64
 5   Cash_ratio                                9000 non-null   float64
 6   Quick_ratio                               9000 non-null   float64
 7   Current_ratio                             9000 non-null   float64
 8   ROA(new)                                  9000 non-null   float64
 9   ROE(new)                                  9000 non-null   float64
 10  ROS(new)                            

## After imputing the column, Shareholderquity_code let us find out the count of values

In [23]:
X_df.loc[X_df['Shareholderquity_code'] < 0.5, 'Shareholderquity_code' ] = 0
X_df.loc[X_df['Shareholderquity_code'] > 3, 'Shareholderquity_code' ] = 3
X_df['Shareholderquity_code']          =  X_df['Shareholderquity_code'].round()
### 
X_df['Shareholderquity_code']  =  X_df['Shareholderquity_code'].astype(str)

In [24]:
X_df['Shareholderquity_code'].value_counts()

0.0    7035
2.0     896
1.0     619
3.0     450
Name: Shareholderquity_code, dtype: int64

### After imputing

| Value | Count Before imputing | Count after imputing |
| ---- | ------- | ------ |
| 0 | 3305 | 7048 |
| 1 | 601 | 606 |
| 2 | 865 | 896 |
| 3 | 439 | 450 |
| Null | 3790 | 0 |

### Observations

All the variables are of float data type. We shall convert the following variables to integer data type:
* 1) Year Encoded
* 2) Target

In [25]:
X_df['Year Encoded']                         =  X_df['Year Encoded'].astype(int)
X_df['Target']                               =  X_df['Target'].astype(int)

In [26]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year Encoded                              9000 non-null   int32  
 1   Target                                    9000 non-null   int32  
 2   Asset_turnover                            9000 non-null   float64
 3   Receivable_turnover(new)                  9000 non-null   float64
 4   Inventory_turnover                        9000 non-null   float64
 5   Cash_ratio                                9000 non-null   float64
 6   Quick_ratio                               9000 non-null   float64
 7   Current_ratio                             9000 non-null   float64
 8   ROA(new)                                  9000 non-null   float64
 9   ROE(new)                                  9000 non-null   float64
 10  ROS(new)                            

### Change the order of data frame

In [27]:
new_cols  = ["Target", "Year Encoded", "Asset_turnover", "Receivable_turnover(new)", "Inventory_turnover", "Cash_ratio",\
       "Quick_ratio", "Current_ratio", "ROA(new)", "ROE(new)", "ROS(new)","ROI(new)", "debt_asset", "debt_equity",\
       "debt_income","Interest_coverage", "Asset_coverage", "EBIT_Sales", "Sales_CE", "ROCE_CE", "Changeinsales_Industry",\
       "Grossvaluedadded", "Ln_GVA", "Operating Cash Flow/Total Sales", "Operating Cash Flow/Total Debt",\
       "Operating Cash Flow/Shareholder's Equity","Fixed Asset Turnover Ratio", "YOY Sales Growth Rate","YOY EBIT Growth Rate",\
       "Total shareholders' funds", 'Shareholderquity_code'] 
df_imputed = X_df[new_cols]

In [28]:
df_imputed.head().T

,0,1,2,3,4
Target,0,0,0,0,0
Year Encoded,-8,-7,-6,-5,-4
Asset_turnover,0.00000,0.75666,1.07063,1.18632,1.42893
Receivable_turnover(new),0.00000,3.46077,3.95393,4.54417,6.10799
Inventory_turnover,0.00000,0.00000,0.00000,0.00000,0.00000
Cash_ratio,2.01764,1.55843,0.85337,0.96842,1.21196
Quick_ratio,1.55202,1.08407,0.38471,0.37537,0.61504
Current_ratio,3.73717,3.03393,2.56104,2.98033,3.18905
ROA(new),0.31478,0.29401,0.38998,0.38976,0.43305
ROE(new),0.39534,0.38944,0.52113,0.49546,0.55367


#### 3.4)  Feature Engineering

*Recursive Feature Elimination, or RFE for short*, is a popular feature selection algorithm in a dataset that are more or most relevant in predicting the target variable.

**RFE** applies a backward selection process to find the best combination of features. This is done as follows:
* 1) Builds a model based on all features and calculates the importance of each feature in the model.
* 2) It ranks the features and removes the feature(s) with the least importance iteratively based on model evaluation
metrics such as accuracy ratio.

Ref. https://towardsdatascience.com/effective-feature-selection-recursive-feature-elimination-using-r-148ff998e4f7

In [29]:
# define dataset
y             =    df_imputed['Target']
X             =    df_imputed.drop(['Target', 'Shareholderquity_code'], axis = 1)                       
print('X dimension {}'. format(X.shape))
print('y dimension {}'. format(y.shape))

X dimension (9000, 29)
y dimension (9000,)


In [30]:
collist =  X.columns

In [31]:
model_L         =    list()
df_imp          =    pd.DataFrame()

In [32]:
# define RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select = 10)
# fit RFE
rfe.fit(X, y)
# summarize all features

sno_L           =    list()
support_text    =    list()
rank_list       =    list()
each            =    'Decision Tree'
print("Proecessing %s" % each)
for i in range(X.shape[1]):
    model_L.append(each)
    sno_L.append(i)
    support_text.append(rfe.support_[i])
    rank_list.append(rfe.ranking_[i])
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    
df_imp_ind1   =    pd.DataFrame({'Sno' : sno_L, 'Model' : model_L, 'Selected?' : support_text, 'Rank' : rank_list, 'Columns' : collist })


Proecessing Decision Tree
Column: 0, Selected False, Rank: 9.000
Column: 1, Selected False, Rank: 14.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected False, Rank: 7.000
Column: 6, Selected False, Rank: 5.000
Column: 7, Selected False, Rank: 11.000
Column: 8, Selected False, Rank: 20.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 12.000
Column: 11, Selected False, Rank: 17.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected True, Rank: 1.000
Column: 14, Selected True, Rank: 1.000
Column: 15, Selected False, Rank: 13.000
Column: 16, Selected False, Rank: 19.000
Column: 17, Selected False, Rank: 6.000
Column: 18, Selected False, Rank: 8.000
Column: 19, Selected True, Rank: 1.000
Column: 20, Selected False, Rank: 2.000
Column: 21, Selected False, Rank: 18.000
Column: 22, Selected False, Rank: 4.000
Column: 23, Selected False, Rank: 15.000
Column: 24, Selected Fa

In [33]:
# define RFE
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select = 10)
# fit RFE
rfe.fit(X, y)

# summarize all features
model_L         =    list()
sno_L           =    list()
support_text    =    list()
rank_list       =    list()
each            =    'Random Forest'
print("Proecessing %s" % each)

for i in range(X.shape[1]):
    model_L.append(each)
    sno_L.append(i)
    support_text.append(rfe.support_[i])
    rank_list.append(rfe.ranking_[i])
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    
df_imp_ind2   =    pd.DataFrame({'Sno' : sno_L, 'Model' : model_L, 'Selected?' : support_text, 'Rank' : rank_list, 'Columns' : collist })


Proecessing Random Forest
Column: 0, Selected False, Rank: 17.000
Column: 1, Selected False, Rank: 7.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected False, Rank: 3.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected False, Rank: 11.000
Column: 7, Selected False, Rank: 6.000
Column: 8, Selected False, Rank: 12.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 13.000
Column: 11, Selected False, Rank: 18.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected True, Rank: 1.000
Column: 14, Selected True, Rank: 1.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected False, Rank: 14.000
Column: 17, Selected False, Rank: 10.000
Column: 18, Selected False, Rank: 5.000
Column: 19, Selected False, Rank: 4.000
Column: 20, Selected False, Rank: 19.000
Column: 21, Selected False, Rank: 9.000
Column: 22, Selected False, Rank: 20.000
Column: 23, Selected True, Rank: 1.000
Column: 24, Selected Fal

In [34]:
# define RFE
rfe = RFE(estimator = LinearDiscriminantAnalysis(solver = 'svd'), n_features_to_select = 10)
# fit RFE
rfe.fit(X, y)

# summarize all features
model_L         =    list()
sno_L           =    list()
support_text    =    list()
rank_list       =    list()
each            =    'LDA'
print("Proecessing %s" % each)

for i in range(X.shape[1]):
    model_L.append(each)
    sno_L.append(i)
    support_text.append(rfe.support_[i])
    rank_list.append(rfe.ranking_[i])
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    
df_imp_ind3   =    pd.DataFrame({'Sno' : sno_L, 'Model' : model_L, 'Selected?' : support_text, 'Rank' : rank_list, 'Columns' : collist })


Proecessing LDA
Column: 0, Selected True, Rank: 1.000
Column: 1, Selected False, Rank: 9.000
Column: 2, Selected False, Rank: 13.000
Column: 3, Selected False, Rank: 10.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected True, Rank: 1.000
Column: 7, Selected False, Rank: 3.000
Column: 8, Selected False, Rank: 5.000
Column: 9, Selected False, Rank: 6.000
Column: 10, Selected False, Rank: 19.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 4.000
Column: 14, Selected False, Rank: 18.000
Column: 15, Selected False, Rank: 17.000
Column: 16, Selected True, Rank: 1.000
Column: 17, Selected False, Rank: 12.000
Column: 18, Selected False, Rank: 7.000
Column: 19, Selected False, Rank: 2.000
Column: 20, Selected False, Rank: 14.000
Column: 21, Selected True, Rank: 1.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 16.000
Column: 24, Selected False, Rank: 8

In [35]:
# Appending multiple DataFrame
df_imp       =    pd.DataFrame()
df_imp       =    pd.concat([df_imp_ind1, df_imp_ind2, df_imp_ind3], ignore_index=True)

In [36]:
df_imp.head().T

,0,1,2,3,4
Sno,0,1,2,3,4
Model,Decision Tree,Decision Tree,Decision Tree,Decision Tree,Decision Tree
Selected?,False,False,True,True,True
Rank,9,14,1,1,1
Columns,Year Encoded,Asset_turnover,Receivable_turnover(new),Inventory_turnover,Cash_ratio


In [37]:
df_important =    df_imp.loc[df_imp['Selected?'] == True]

In [38]:
df_important.shape

(30, 5)

In [39]:
df_important

,Sno,Model,Selected?,Rank,Columns
2,2,Decision Tree,True,1,Receivable_turnover(new)
3,3,Decision Tree,True,1,Inventory_turnover
4,4,Decision Tree,True,1,Cash_ratio
9,9,Decision Tree,True,1,ROS(new)
12,12,Decision Tree,True,1,debt_equity
13,13,Decision Tree,True,1,debt_income
14,14,Decision Tree,True,1,Interest_coverage
19,19,Decision Tree,True,1,Changeinsales_Industry
25,25,Decision Tree,True,1,Fixed Asset Turnover Ratio
28,28,Decision Tree,True,1,Total shareholders' funds


In [40]:
parent_dir

'D:\\DrPKV\\20220901'

In [41]:
path   =    parent_dir + r'\Output_' + datetime.now().strftime("%Y%m%d%I%M%S%p")
print(path)
os.mkdir(path)

D:\DrPKV\20220901\Output_20220901092149AM


In [42]:
important_features_file =  path +  r'\Important_Features_Bankruptcy_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
df_important.to_csv(important_features_file, index = False)

In [43]:
imp_cols_list =   df_important['Columns'].tolist()

In [44]:
ul = np.unique(imp_cols_list).tolist()

In [45]:
print(ul); print(len(ul))

['Asset_coverage', 'Cash_ratio', 'Changeinsales_Industry', 'Current_ratio', 'EBIT_Sales', 'Fixed Asset Turnover Ratio', 'Interest_coverage', 'Inventory_turnover', 'Ln_GVA', 'Operating Cash Flow/Total Debt', 'Operating Cash Flow/Total Sales', 'Quick_ratio', 'ROS(new)', 'Receivable_turnover(new)', "Total shareholders' funds", 'YOY EBIT Growth Rate', 'Year Encoded', 'debt_asset', 'debt_equity', 'debt_income']
20


### Observations

We applied three models to select important features 

Twenty columns are idemtified as important features in predicting the target variable (Bankruptcy).

They are as follows:

| S No | Column            | Remarks                                          |
|:--- | --------------------------- | ----------------------------------- |
| 1 | Asset_coverage | Selected by Random Forest model |
| 2 | Cash_ratio | Selected by Decision Tree (CART), Random Forest, LDA models | 
| 3 | Changeinsales_Industry | Selected by Decision Tree (CART),Random Forest models| 
| 4 | Current_ratio | Selected by LDA model | 
| 5 | debt_asset | Selected by LDA model | 
| 6 | debt_equity | Selected by Decision Tree (CART),Random Forest & LDA models| 
| 7 | debt_income| Selected by Decision Tree (CART) & Random Forest models |
| 8 | EBIT_Sales | Selected by LDA model | 
| 9 | Fixed Asset Turnover Ratio | Selected by Decision Tree (CART) model |
| 10 | Interest_coverage | Selected by Decision Tree & Random Forest models |
| 11 | Inventory_turnover | | Selected by Decision Tree (CART) model |
| 12 | Ln_GVA | Selected by LDA model | 
| 13 | Operating Cash Flow/Total Debt | Selected by Random Forest model |
| 14 | Operating Cash Flow/Total Sales | Selected by LDA model | 
| 15 | Quick_ratio | Selected by LDA model |
| 16 | Receivable_turnover(new) | Selected by Decision Tree & Random Forest models |
| 17 | ROS(new) | Selected by Decision Tree (CART),Random Forest models | 
| 18 | Total shareholders' funds | Selected by Decision Tree (CART),Random Forest models|
| 19 | Year Encoded | Selected by LDA model | 
| 20 | YOY EBIT Growth Rate | Selected by LDA model | 



In [46]:
all_important_features_file = path +  r'\imputed_dataset_all_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
df_imputed.to_csv(all_important_features_file, index = False)

In [47]:
imp_cols_list.insert(0, 'Target')
df_imporant_20    =  df_imputed[imp_cols_list]

In [48]:
df_imporant_20.head()

,Target,Receivable_turnover(new),Inventory_turnover,Cash_ratio,ROS(new),debt_equity,debt_income,Interest_coverage,Changeinsales_Industry,Fixed Asset Turnover Ratio,...,Year Encoded,Cash_ratio,Quick_ratio,Current_ratio,debt_asset,debt_equity,EBIT_Sales,Ln_GVA,Operating Cash Flow/Total Sales,YOY EBIT Growth Rate
0,0,0.00000,0.00000,2.01764,10.92376,0.01399,0.03539,0.00000,0.11152,0.00000,...,-8,2.01764,1.55202,3.73717,0.01114,0.01399,817.98315,14.12832,817.97405,-0.30656
1,0,3.46077,0.00000,1.55843,0.28722,0.01663,0.04271,-9600.52367,0.11152,2.77352,...,-7,1.55843,1.08407,3.03393,0.01256,0.01663,0.38857,14.15909,0.41343,-0.00279
2,0,3.95393,0.00000,0.85337,0.28324,0.01936,0.03716,-3777.26754,0.11152,2.90813,...,-6,0.85337,0.38471,2.56104,0.01449,0.01936,0.36426,14.21219,0.39055,-0.02974
3,0,4.54417,0.00000,0.96842,0.29300,0.02084,0.04207,2887.88607,0.11152,3.13408,...,-5,0.96842,0.37537,2.98033,0.01640,0.02084,0.32854,14.26065,0.35407,0.14724
4,0,6.10799,0.00000,1.21196,0.30306,0.03462,0.06253,11940.01506,0.11152,3.99127,...,-4,1.21196,0.61504,3.18905,0.02708,0.03462,0.30306,14.33665,0.32075,-0.22265


In [49]:
Twenty_important_features_file =  path + '\imputed_dataset_20_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
df_imporant_20.to_csv(Twenty_important_features_file, index = False)

By majority votes, the following columns are identified as important by at least two models

* 1.  Cash_ratio
* 2.  Changeinsales_Industry
* 3.  debt_equity
* 4.  debt_income
* 5.  Interest_coverage
* 6.  Operating Cash Flow/Total Debt
* 7.  Receivable_turnover(new)
* 8.  ROS(new)
* 9.  Total shareholders' funds

In [50]:
#Shareholderquity_code
df_Orig.columns

Index(['Row', 'Year', 'Company_name', 'Year Encoded',
       ' Dummy Coded :Healthy=1; NPA= 0', 'Asset_turnover',
       'Receivable_turnover(new)', 'Inventory_turnover', 'Cash_ratio',
       'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)', 'ROS(new)',
       'ROI(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       'Operating Cash Flow/Shareholder's Equity',
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', 'Total shareholders' funds',
       'Shareholderquity_code'],
      dtype='object')

In [51]:
consensus_cols = ['Cash_ratio', 'Changeinsales_Industry','debt_equity', 'debt_income','Interest_coverage',\
                  'Operating Cash Flow/Total Debt','Receivable_turnover(new)','ROS(new)',"Total shareholders' funds"]

df_imporant_10  =  df_imputed[consensus_cols]

In [52]:
nrows                                     =    df_imputed.shape[0] 
df_imporant_10['Slno']                    =    range(nrows)
df_imporant_10['Shareholderquity_code']   =    df_imputed['Shareholderquity_code'].astype(str)
df_imporant_10['Target']                  =    df_imputed['Target']
### -------------------------------------- --------------------------------

C:\Users\User\AppData\Local\Temp\ipykernel_12028\1814212724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imporant_10['Slno']                    =    range(nrows)
C:\Users\User\AppData\Local\Temp\ipykernel_12028\1814212724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imporant_10['Shareholderquity_code']   =    df_imputed['Shareholderquity_code'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_12028\1814212724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [53]:
df_imporant_10.columns

Index(['Cash_ratio', 'Changeinsales_Industry', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Operating Cash Flow/Total Debt',
       'Receivable_turnover(new)', 'ROS(new)', 'Total shareholders' funds',
       'Slno', 'Shareholderquity_code', 'Target'],
      dtype='object')

In [54]:
df_imporant_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Cash_ratio                      9000 non-null   float64
 1   Changeinsales_Industry          9000 non-null   float64
 2   debt_equity                     9000 non-null   float64
 3   debt_income                     9000 non-null   float64
 4   Interest_coverage               9000 non-null   float64
 5   Operating Cash Flow/Total Debt  9000 non-null   float64
 6   Receivable_turnover(new)        9000 non-null   float64
 7   ROS(new)                        9000 non-null   float64
 8   Total shareholders' funds       9000 non-null   float64
 9   Slno                            9000 non-null   int64  
 10  Shareholderquity_code           9000 non-null   object 
 11  Target                          9000 non-null   int32  
dtypes: float64(9), int32(1), int64(1),

In [55]:
df_imporant_10.head().T

,0,1,2,3,4
Cash_ratio,2.01764,1.55843,0.85337,0.96842,1.21196
Changeinsales_Industry,0.11152,0.11152,0.11152,0.11152,0.11152
debt_equity,0.01399,0.01663,0.01936,0.02084,0.03462
debt_income,0.03539,0.04271,0.03716,0.04207,0.06253
Interest_coverage,0.00000,-9600.52367,-3777.26754,2887.88607,11940.01506
Operating Cash Flow/Total Debt,29.35541,24.91447,28.85392,25.61557,16.92720
Receivable_turnover(new),0.00000,3.46077,3.95393,4.54417,6.10799
ROS(new),10.92376,0.28722,0.28324,0.29300,0.30306
Total shareholders' funds,1439044420.00000,1012901796.00000,754886314.00000,770367465.00000,790918031.00000
Slno,0,1,2,3,4


### Split data into training and test datasets

In [56]:
y                                 =  df_imporant_10['Target']
X                                 =  df_imporant_10.drop(['Target'], axis = 1)   
###

random_state                      =  1234
X_train, X_test, y_train, y_test  =  train_test_split(X, y, test_size = 0.30, stratify = y,  random_state = random_state)  
print("X_train Dataset Dimension {}".format(X_train.shape))
print("y_train Dataset Dimension {}".format(y_train.shape))
print("X_test  Dataset Dimension {}".format(X_test.shape))
print("y_test  Dataset Dimension {}".format(y_test.shape))

X_train Dataset Dimension (6300, 11)
y_train Dataset Dimension (6300,)
X_test  Dataset Dimension (2700, 11)
y_test  Dataset Dimension (2700,)


In [57]:
X_train_10_important_features_file = path + r'\X_train_compact_dataset_10_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
X_train.to_csv(X_train_10_important_features_file, index = False)                               

In [58]:
y_train_10_important_features_file =  path + r'\y_train_compact_dataset_10_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
y_train.to_csv(y_train_10_important_features_file, index = False)                               

In [59]:
X_test_10_important_features_file  =  path + r'\X_test_compact_dataset_10_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
X_test.to_csv(X_test_10_important_features_file, index = False)                               

In [60]:
y_test_10_important_features_file  =  path + r'\test_compact_dataset_10_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
y_test.to_csv(y_test_10_important_features_file, index = False)                               

In [61]:
ten_important_features_file =  path + r'\imputed_compact_dataset_10_' + datetime.now().strftime("%Y%m%d%I%M%S%p") + '.csv'
df_imporant_10.to_csv(ten_important_features_file, index = False)                               

### END